# dataset generator

In [56]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import numpy as np

In [62]:
from glob import glob
import uuid

In [191]:
fonts = glob('fonts/*')
val_font = fonts[4]
fonts = fonts[:4]

In [193]:
symbols = ['a', 'b', 'v', 'x']

In [202]:
def gen_one_symbol(c:str, font_path:str, dataset_path:str) -> dict[str: str]:
    id_str = str(uuid.uuid4())
    path = f'{dataset_path}/{id_str}.png'
    w,h = 50,50
    point = w//2-15, h//2-20
    image = np.zeros((w,h),dtype=np.uint8)
    image = Image.fromarray(image)
    draw = ImageDraw.Draw(image)
    font_size = 50
    font = ImageFont.truetype(font_path, font_size)
    draw.text(point, c, font=font,fill=(255), align='center')
    image.save(path)
    return {'label':c, 'path': path}

In [192]:
dataset = []
for c in symbols:
    for font in fonts:
        dataset.append(gen_one_symbol(c,font, 'dataset'))

NameError: name 'symbols' is not defined

In [210]:
dataset_val = []
for c in symbols:
    dataset_val.append(gen_one_symbol(c,val_font, 'dataset_validate'))

In [207]:
from PIL import ImageFilter

def crop(image: np.ndarray):
    cols = np.where(image.mean(axis=0) > 10)
    x0,x1=np.min(cols), np.max(cols)
    rows = np.where(image.mean(axis=1) > 10)
    y0,y1=np.min(rows), np.max(rows)
    return image[y0:y1+1,x0:x1+1]
def to_shape(cropped: np.ndarray, image_size = 28):
    img_cr = Image.fromarray(cropped)
    return img_cr.resize((image_size, image_size), Image.LANCZOS)

def aug(image, label, dataset_path):
    meta = [{'label': label, 'path': f'{dataset_path}/{uuid.uuid4()}.png'} for cl in range(4)]
    image.filter(ImageFilter.EDGE_ENHANCE).save(meta[0]['path'])
    image.filter(ImageFilter.BoxBlur(3)).save(meta[1]['path'])
    image.effect_spread(2).save(meta[2]['path'])
    image.save(meta[3]['path'])
    return meta
    

In [211]:
aug_dataset = []
for row in dataset:
    img = Image.open(row['path'])
    img = np.asarray(img)
    img = crop(img)
    img = to_shape(img)
    aug_dataset.extend(aug(img, row['label']))

KeyError: 'path'

In [216]:
aug_dataset_validate = []
for row in dataset_val:
    img = Image.open(row['path'])
    img = np.asarray(img)
    img = crop(img)
    img = to_shape(img)
    aug_dataset_validate.extend(aug(img, row['label'], 'aug_dataset_validate'))

In [194]:
import pandas as pd

In [196]:
pd.DataFrame(aug_dataset).to_csv("train.csv")

In [217]:
pd.DataFrame(aug_dataset_validate).to_csv("val.csv")